<a href="https://colab.research.google.com/github/DarioRugg/KickLearning/blob/main/feature_analysis/Text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
![ ! -f "translate.py" ] && wget translate.py https://raw.githubusercontent.com/DarioRugg/KickLearning/main/feature_analysis/translate.py
!pip install pycld2 sentencepiece transformers polyglot pyicu pySBD

In [ ]:
filename = 'file_0000_scraped.csv'

In [ ]:
!python translate.py $filename